# Assigment: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [12]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms

In [13]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [14]:
from torchvision import models

resnet = models.resnet18(pretrained=False)

In [15]:
device = torch.device('cuda')
for images, labels in trainset:
  images.to(device)
for images, labels in testset:
  images.to(device)

In [16]:
model = resnet
model.to(device) # transfer model to GPU
model.train()    # set to train mode
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.003, momentum=0.9)

In [17]:
 def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)


In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, criterion, optimizer)
    test(testloader, model, criterion)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 43.4%, Avg loss: 1.557075 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.443280 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 56.2%, Avg loss: 1.270240 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.221060 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.178969 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 63.2%, Avg loss: 1.093073 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 66.5%, Avg loss: 0.970544 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.948059 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.068817 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 68.4%, Avg loss: 0.921941 

Done!


## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs

In [9]:
model2 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


In [10]:
model2.to(device) # transfer model to GPU
model2.train()    # set to train mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model2, criterion, optimizer)
    test(testloader, model2, criterion)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 27.084690 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 25.553804 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 28.197449 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 25.972890 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 27.572026 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 25.807919 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 24.829551 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 29.590491 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 29.136974 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 0.1%, Avg loss: 27.499760 

Done!
